In [1]:
from os import listdir
from summa.summarizer import summarize
import stemmsk
import rouge
import nltk
nltk.download('punkt')

DATA_DIR = "data/test_summs/"

[nltk_data] Downloading package punkt to /home/luk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def stem_sentence(sentence):
    new_sentence = []
    for word in sentence.split(" "):
        new_sentence.append(stemmsk.stem(word))
    return " ".join(new_sentence)

In [3]:
stem_sentence("listina základních práv európskej únie")

'list základn práv európsk úni'

In [4]:
text = """Automatic summarization is the process of reducing a text document with a \
computer program in order to create a summary that retains the most important points \
of the original document. As the problem of information overload has grown, and as \
the quantity of data has increased, so has interest in automatic summarization. \
Technologies that can make a coherent summary take into account variables such as \
length, writing style and syntax. An example of the use of summarization technology \
is search engines such as Google. Document summarization is another."""

In [5]:
print(summarize(text))

Automatic summarization is the process of reducing a text document with a computer program in order to create a summary that retains the most important points of the original document.


In [10]:
files = listdir(DATA_DIR)
print(len(files))

print(files[0])
with open(DATA_DIR + files[0], 'r') as r:
    whole = r.read().split('\n')[:-1]
    summ, text = whole
    
print(summ + '\n')
print(text)

12256
20a1cb1e013b3efbbf19409a870dd004875dc3c8.html
Centy v peňaženkách Slovákov zatiaľ ostanú Ilustračné foto

Centy zatiaľ Slovákom v peňaženkách ostanú . Na dnešnej konferencii venovanej prvému roku s eurom to povedal guvernér Národnej banky Slovenska ( NBS ) Ivan Šramko . Podľa prieskumu , ktorý realizovalo Zastúpenie Európskej komisie na Slovensku v októbri tohto roku , vyplýva , že spotrebiteľom aj obchodníkom z euromincí najviac " prekážajú " práve jedno - a dvojcentové mince . " Rušenie centov ako téma neexistuje . Občania si zvykli , pokiaľ niekto nemá záujem ich používať , tak ich nepoužíva , " konštatoval Šramko . Prezident Zväzu obchodu a cestovného ruchu Pavol Konštiak si naopak myslí , že by bolo vhodné , aby sa začalo diskutovať aspoň o možnosti zaokrúhľovania cien na 5 centov . " Zákon o zaokrúhľovaní na päťcentovky by výrazne pomohol spotrebiteľom , ale aj obchodníkom , " spresnil . Centy by tak nezmizli , používal by ich len ten , kto chce a ceny by boli podľa Konštia

------------------------------------------------------------------------------------------------------------------

In [7]:
evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l'],
                           max_n=2,
                           limit_length=True,
                           length_limit=100,
                           length_limit_type='words',
                           apply_avg=True,
                           apply_best=False,
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=False)

print('f means f1 score, r means recall, p means precision\n')
for i in range(3):
    with open(DATA_DIR + files[i], 'r') as r:
        whole = r.read().split('\n')[:-1]
        summ, text = whole
        tr = summarize(text).replace('\n', ' ')
        
        print('TARGET: ' + summ)
        print('TEXT RANK: ' + tr)
        
        scores = evaluator.get_scores([tr], [summ])
        print(f"rouge-l f1 score: {scores['rouge-l']['f']} \n")

f means f1 score, r means recall, p means precision

TARGET: Centy v peňaženkách Slovákov zatiaľ ostanú Ilustračné foto
TEXT RANK: Prezident Zväzu obchodu a cestovného ruchu Pavol Konštiak si naopak myslí , že by bolo vhodné , aby sa začalo diskutovať aspoň o možnosti zaokrúhľovania cien na 5 centov .
rouge-l f1 score: 0.07307837217635979 

TARGET: V USA skrachovalo ďalších 8 bánk , celkovo v tomto roku už 118 Americké regulačné úrady zatvorili ďalších 8 bánk . Celkovo ich v tomto roku v USA skrachovalo už 118 . Počet bankrotov amerických bánk sa zvyšuje a v tomto roku zrejme prekoná vlaňajších 140 , uviedla šéfka Federálneho úradu na ochranu vkladov FDIC Sheila Bairová v júli v rozhovore pre Bloomberg Television . Pobočka skrachovanej banky ShoreBank
TEXT RANK: Väčšinou ide o menšie banky , s výnimkou chicagskej ShoreBank , ktorá je s aktívami v hodnote 2 , 16 miliardy USD ( 1 , 70 miliardy eur ) a vkladmi vo výške 1 , 54 miliardy USD najväčším americkým finančným inštitútom zbankroto

In [11]:
summarizations = []
hypotheses = []

for i in range(len(files)):
    with open(DATA_DIR + files[i], 'r') as r:
        whole = r.read().split('\n')[:-1]
        summ, text = whole
        tr = summarize(text).replace('\n', ' ')
        summarizations.append(stem_sentence(summ))
        hypotheses.append(stem_sentence(tr))
        
        if i % 1000 == 0:
                print(f"{i}/{len(files)}")


scores = evaluator.get_scores(hypotheses, summarizations)
for metric in scores.keys():
    results = scores[metric]
    print(f"{metric} avg f1 score: {results['f']}")

0/12256
1000/12256
2000/12256
3000/12256
4000/12256
5000/12256
6000/12256
7000/12256
8000/12256
9000/12256
10000/12256
11000/12256
12000/12256
rouge-1 avg f1 score: 0.19922429806889883
rouge-2 avg f1 score: 0.05733369965238888
rouge-l avg f1 score: 0.18331173488840521
